In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.models import load_model

import os

In [34]:
PATH = "../input/dermmel/DermMel/"

PATH_TRAIN = PATH + '/train_sep/'
PATH_VALID = PATH + '/valid/'
PATH_TEST = PATH + '/test/'

In [35]:
def dataset(path, type):
    labels = os.listdir(path)
    total = 0
    for label in labels:
        print(label,len(os.listdir(path + label)))
        total += len(os.listdir(path + label))

    print(type,"photos",total)
    print ("\n")

dataset(PATH_TRAIN,"training")
dataset(PATH_VALID,"validation")
dataset(PATH_TEST,"testing")

In [54]:
def dataset_display(path, sample, type):
    img_path = path + '/' + type + '/'
    img_name = os.listdir(img_path)[sample]
    img_path_full = img_path + img_name
    img = load_img(img_path_full, target_size=(224, 224))
    imgplot = plt.imshow(img)
    print(img_path, "image number", sample)
    plt.show()
    return img_path_full

dataset_display(PATH_TRAIN, 77, 'Melanoma')
dataset_display(PATH_TRAIN, 77, 'NotMelanoma')
dataset_display(PATH_TEST, 77, 'Melanoma')
dataset_display(PATH_TEST, 77, 'NotMelanoma')
dataset_display(PATH_VALID, 77, 'Melanoma')
dataset_display(PATH_VALID, 77, 'NotMelanoma')

In [5]:
conv_base = ResNet50(weights='imagenet',
                     include_top=False,
                     input_shape=(224, 224, 3))

In [6]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

print(model.summary())

In [7]:
#Make the conv_base not trainable:
for layer in conv_base.layers[:]:
    layer.trainable = False

In [8]:
model.compile(optimizer=optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

In [9]:
batch_size = 20
target_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(PATH_TRAIN,
                                              target_size=target_size,
                                              batch_size=batch_size)

valid_gen = test_datagen.flow_from_directory(PATH_VALID,
                                             target_size=target_size,
                                             batch_size=batch_size)

test_gen = test_datagen.flow_from_directory(PATH_TEST,
                                            target_size=target_size,
                                            batch_size=batch_size)

In [10]:
history = model.fit_generator(train_gen,
                              epochs=1,
                              steps_per_epoch = 4096 // batch_size,
                              validation_data = valid_gen,
                              validation_steps = 2048 // batch_size)

In [11]:
# make last block of the conv_base trainable:

for layer in conv_base.layers[:165]:
    layer.trainable = False
for layer in conv_base.layers[165:]:
    layer.trainable = True

In [12]:
model.compile(optimizer=optimizers.Adam(lr=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
history = model.fit_generator(train_gen,
                              epochs=1,
                              steps_per_epoch = 4096 // batch_size,
                              validation_data = valid_gen,
                              validation_steps = 2048 // batch_size)

In [14]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_gen, steps = 2048 // batch_size, verbose=1)
print('test accuracy:', test_acc)
print('test loss:', test_loss)

In [93]:
def load_image(img_path_full, show = False):
    img = image.load_img(img_path_full, target_size = (224, 224))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis = 0)
    img_tensor /= 255
    
    return img_tensor

check_type = 'NotMelanoma'

CHECK_PATH = dataset_display(PATH_TEST, 88, check_type)
pred = model.predict(load_image(CHECK_PATH))
pred = round(float(pred[0][0]),4)*100
actual = 100 if check_type == 'Melanoma' else 0
print (f'Predicted chance of melanoma: {pred}%')
print (f"Actual: {actual}%")
print (f'Diff (lower is better): {abs(round(pred-actual,4))}%')

In [62]:
model.save('melanoma_resnet.h5')